In [3]:
import pandas as pd
df = pd.read_csv('.csv', encoding='cp949')

In [ ]:
# 컬럼별 중복 제거 개수 확인
columns = df.columns
columns_info = dict()

for col in columns:
    columns_info[col] = df[col].nunique()
columns_info

In [ ]:
# 결측치 확인 - 여러 컬럼에서 결측치 확인

null_cnt_df = pd.DataFrame(df.isnull().sum()).rename(columns={0: 'null_count'}).reset_index()
null_cnt_df['null_ratio'] = round(null_cnt_df['null_count'] / len(df) *100,2)
null_cnt_df

과목코드 결측치 채우기

In [ ]:
# 학년 열의 뒷자리 추출
df['학년_뒷자리'] = df['학년'].str[-1]

# 각 학년 열의 뒷자리에 해당하는 학년 값의 빈도 계산
과목코드_frequency_by_grade = df.groupby('학년_뒷자리')['학년'].value_counts().reset_index(name='frequency')

과목코드_frequency_by_grade

In [7]:
# 각 grade_cd_last_digit별 최빈 mcode 값 계산
mode_by_grade = 과목코드_frequency_by_grade.groupby('학년_뒷자리')['과목코드'].apply(lambda x: x.mode()[0]).reset_index(name='과목코드')

# mcode 열의 결측치를 학년별 최빈 과목으로 채우기
for index, row in mode_by_grade.iterrows():
    grade_digit = row['학년_뒷자리']
    mode_mcode = row['과목코드']
    df.loc[(df['학년_뒷자리'] == grade_digit) & (df['과목코드'].isnull()), '과목코드'] = mode_mcode

In [ ]:
# 결측치 확인

null_cnt_df = pd.DataFrame(df.isnull().sum()).rename(columns={0: 'null_count'}).reset_index()
null_cnt_df['null_ratio'] = round(null_cnt_df['null_count'] / len(df) *100,2)
null_cnt_df

# 그 외 결측치 존재하는 열은 정규분포 여부 확인 후 ZSCORE/IQR 사용하여 결측치 처리

In [26]:
from scipy.stats import shapiro

# Shapiro-Wilk 테스트를 사용하여 정규성 검정
statistic, p_value = shapiro(df[''])

# p-value를 기준으로 정규성 여부 판단
if p_value > 0.05:
    print("데이터가 정규분포를 따릅니다.")
    # 여기에 이상치 처리 등을 추가할 수 있습니다.
else:
    print("데이터가 정규분포를 따르지 않습니다.")


데이터가 정규분포를 따르지 않습니다.


In [ ]:
# 사분위수 계산
Q1 = df[''].quantile(0.25)
Q3 = df[''].quantile(0.75)
IQR = Q3 - Q1

# 사분위수를 기반으로 이상치 선택
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

selected_outliers = df[(df[''] < lower_bound) | (df[''] > upper_bound)]
selected_outliers 

In [ ]:
# 이상치를 중앙값으로 대체
median_value = df[''].median()
df.loc[(df[''] < lower_bound) | (df[''] > upper_bound), ''] = median_value

In [52]:
# 결측치 비율이 모두 1퍼센트 미만 행 삭제
# 선택된 열들에 대해 결측치가 있는 행 삭제
cleaned_df = df.dropna(subset=[''])

# 새로운 데이터프레임을 기존의 df에 할당
df = cleaned_df.copy()

In [ ]:
# 최종 전처리된 데이터 결측치 확인
null_cnt_df = pd.DataFrame(df.isnull().sum()).rename(columns={0: 'null_count'}).reset_index()
null_cnt_df['null_ratio'] = round(null_cnt_df['null_count'] / len(df) *100,2)
null_cnt_df

In [75]:
df.to_excel('processed_data.xlsx', index=False)

인코딩

목표 10 -> 0, 30 -> 1로 변환

In [124]:
# Target 변환
df['목표'] = df['목표'].apply(lambda x: 0 if x == 10 else 1)

성별 인코딩

In [ ]:
# 성별 열을 새로운 열로 추가하고 인코딩
df['성별'] = df['성별구분'].replace({'남': 0, '여': 1, '정보없음': 3})

# 결과 확인
print(df[['성별구분', '성별']].head(10))


과목코드 인코딩

In [ ]:
# 국어를 0, 영어를 1, 수학을 2로 매핑하고 그 외를 3으로 지정
df['과목'] = df['과목코드'].map({'국어': 0, '영어': 1, '수학': 2}).fillna(3).astype(int)

# 결과 확인
print(df[['과목코드', '과목']].head(10))

In [147]:
# 'N'을 0, 'Y'를 1로 매핑하여 인코딩
df['A'] = df['A'].map({'N': 0, 'Y': 1})
df['B'] = df['B'].map({'N': 0, 'Y': 1})
df['C'] = df['C'].map({'N': 0, 'Y': 1})

시각화

In [83]:
import matplotlib as mpl
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

sns.set(font_scale=1.4)
# 맑은 고딕으로 글꼴 설정하기
plt.rc('font', family='Malgun Gothic') 
# 마이너스 기호 문제 해결하기
mpl.rcParams['axes.unicode_minus'] = False

train, test데이터 분리

In [156]:
# 관련 없는 열 분리
x=df.drop([''], axis=1)
y=df['목표']

In [157]:
from sklearn.model_selection import train_test_split

# Train, test 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

수치형에 대해 min-max 스케일링

In [158]:
from sklearn.preprocessing import MinMaxScaler

# Min-Max 스케일러 생성
scaler = MinMaxScaler()

# 수치형 열 선택
numeric_cols = ['']

# Train 데이터에 Min-Max 스케일링 적용
X_train[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])

# Test 데이터에는 학습된 스케일러를 사용하여 스케일링 적용
X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])

의사결정나무

In [172]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# 의사결정나무 모델 초기화
clf = DecisionTreeClassifier(max_depth=3,random_state=42)

# 모델 학습
clf.fit(X_train, y_train)

# 예측
y_pred_train = clf.predict(X_train)
y_pred_test = clf.predict(X_test)

# 정확도 출력
train_accuracy = accuracy_score(y_train, y_pred_train)
test_accuracy = accuracy_score(y_test, y_pred_test)

print(f"Train Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")


Train Accuracy: 0.9598
Test Accuracy: 0.9571


Train 정확도와 Test 정확도가 모두 높은 수준으로 나타남

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree
import matplotlib.pyplot as plt

# 의사결정나무 모델 생성
clf = DecisionTreeClassifier(max_depth=3)
clf.fit(X_train, y_train)

# 트리 시각화
plt.figure(figsize=(20,10))
plot_tree(clf, filled=True, feature_names=X_train.columns, class_names=['0', '1'])
plt.show()


특성 중요도

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 특성 중요도 추출
feature_importance = clf.feature_importances_

# 특성명과 중요도를 데이터프레임으로 변환
feature_importance_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': feature_importance})

# 중요도 순으로 정렬
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# 중요도 표시
print(feature_importance_df)

# 중요도 정렬
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=True)


# 중요도 시각화
plt.figure(figsize=(10, 6))
plt.barh(feature_importance_df['Feature'], feature_importance_df['Importance'])
plt.xlabel('Importance')
plt.title('Feature Importance')
plt.show()


In [199]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler()

In [200]:
X_ros, y_ros = ros.fit_resample(X_train, y_train)

In [201]:
from sklearn import tree
DT = tree.DecisionTreeClassifier()
DT.fit(X_ros, y_ros)
y_pred=DT.predict(X_test)

In [202]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, classification_report

In [203]:
print(accuracy_score(y_test, y_pred))

print(precision_score(y_test, y_pred, average='macro'))

print(recall_score(y_test, y_pred, average='macro'))

0.9209932279909706
0.5188368917576961
0.5188368917576961


In [205]:
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.96      0.96      0.96       848
           1       0.08      0.08      0.08        38

    accuracy                           0.92       886
   macro avg       0.52      0.52      0.52       886
weighted avg       0.92      0.92      0.92       886



In [207]:
DT.predict_proba(X_test)

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])